In [134]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.io as pio
import pandas as pd
from pathlib import Path
import datetime
pio.renderers.default = 'iframe'

data_folder = Path('data/')

In [3]:
# remove test station, only look at first half of 2023 to keep the data dimensionality manageable
bike_df = pd.read_parquet(data_folder/'dublinbikes/combined_data_weather_population.parquet', filters=[
    ('time', '>=', datetime.datetime(2023,1,1)),
    ('time', '<=', datetime.datetime(2023,6,1)),
    ('station_id', '!=', 507)
])
bike_df['half_hour'] = bike_df['time'].dt.minute//30*30 == 30
bike_df['empty_station'] = bike_df['available_bikes'] == 0
bike_df['full_station'] = bike_df['available_bike_stands'] == 0
bike_df['needs_attention'] = bike_df['full_station'] | bike_df['empty_station']
bike_df = bike_df.loc[(~pd.isna(bike_df['rain'])) & (~bike_df['half_hour'])] 

print(bike_df.shape)
bike_df.head()

(411250, 29)


,station_id,time,last_updated,name,bike_stands,available_bike_stands,available_bikes,status,address,latitude,...,day_of_month,hour_of_day,rain,temp,rhum,population,half_hour,empty_station,full_station,needs_attention
0,1,2023-01-01 00:00:03,2022-12-31 23:59:39,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,0,0.3,6.8,91.0,5522.0,False,True,False,True
2,1,2023-01-01 01:00:03,2023-01-01 00:50:23,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,1,0.0,6.2,86.0,5522.0,False,True,False,True
4,1,2023-01-01 02:00:03,2023-01-01 01:50:55,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,2,0.0,4.5,91.0,5522.0,False,True,False,True
6,1,2023-01-01 03:00:03,2023-01-01 02:51:28,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,3,0.1,4.0,93.0,5522.0,False,True,False,True
8,1,2023-01-01 04:00:02,2023-01-01 03:52:00,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,4,0.0,2.5,96.0,5522.0,False,True,False,True


In [133]:
app = dash.Dash(__name__)


@app.callback(
    Output('map', 'figure'),
    [Input('time-selector', 'value')] ,
    [State('map', 'relayoutData')]
)
def update_map(selected_time, map_settings):
    selected_time = datetime.datetime.fromtimestamp(selected_time) 
    selected_data = bike_df[bike_df['time'].between(selected_time - datetime.timedelta(minutes=10), selected_time+datetime.timedelta(minutes=10))]
    
    fig = px.scatter_mapbox(selected_data, lat='latitude', lon='longitude', size='available_bikes', hover_data=['station_id', 'address', 'available_bikes', 'bike_stands'], mapbox_style='carto-positron')
    
    lat_range = [selected_data['latitude'].min(), selected_data['latitude'].max()]
    lon_range = [selected_data['longitude'].min(), selected_data['longitude'].max()]
    
    if map_settings and 'mapbox.center' in map_settings:
        center_lat = map_settings['mapbox.center']['lat']
        center_lon = map_settings['mapbox.center']['lon']
    else:
        center_lat = (lat_range[0] + lat_range[1]) / 2
        center_lon = (lon_range[0] + lon_range[1]) / 2 
        
    if map_settings and 'mapbox.zoom' in map_settings:
        zoom_level = map_settings['mapbox.zoom']
    else:
        zoom_level = 12
        
    fig.update_layout(mapbox=dict(center=dict(lat=center_lat, lon=center_lon), zoom=zoom_level),  margin=dict(l=0, r=0, t=0, b=0))  
    
    return fig
    
@app.callback(
    Output('ts', 'figure'),
    [Input('station-selector', 'value'),
     Input('time-selector', 'value')],
)
def update_ts(station, selected_time):
    selected_time = datetime.datetime.fromtimestamp(selected_time)
    start_of_week = selected_time - datetime.timedelta(days=selected_time.weekday())
    start_of_week = start_of_week.replace(hour=0, minute=0, second=0, microsecond=0)
    
    selected_data = bike_df[(bike_df['station_id'] == station) & (bike_df['time'] <= selected_time) & (bike_df['time'] >= start_of_week)]
    fig = px.line(selected_data, x='time', y='available_bikes', title=f'Available Bikes WTD')
    return fig

In [135]:
min_timestamp = bike_df['time'].min().timestamp()
max_timestamp = bike_df['time'].max().timestamp()
        
app.layout = html.Div(children=[
    html.H1(children="Dublin Bikes"),
        
    html.Label('Select Station:'),
    dcc.Dropdown(
        id='station-selector',
        options=[{'label': address, 'value': id} for address, id in zip(bike_df['address'].unique(), bike_df['station_id'].unique())],
        value=bike_df['station_id'].unique()[0]
    ),
        
    html.Label('Select Date and Time:'),
    dcc.Slider(
        id='time-selector',
        step=3600, 
        min=min_timestamp,
        max=max_timestamp,
        marks={i: {'label': pd.to_datetime(i, unit='s').strftime('%Y-%m-%d %H:%M')} for i in range(int(min_timestamp), int(max_timestamp)+1, 3600)},
        value=min_timestamp
    ),

    dcc.Graph(
        id='ts'
    ),
        
    dcc.Graph(
        id='map', 
    ),
    
])

In [136]:
app.run_server(debug=True, use_reloader=False, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/
